In [2]:
import findspark

In [3]:
findspark.init('C:\SPARK')

In [4]:
from pyspark import SparkContext

In [5]:
sc = SparkContext(master = 'local')
sc

<SparkContext master=local appName=pyspark-shell>

In [7]:
print(sc.version)
print(sc.sparkUser())
print(sc.appName)

3.5.0
umutk
pyspark-shell


**Don't forget to close the session**

In [8]:
# Close the session.
sc.stop()

In [13]:
# Ayarlarini manuel olarak yaptigimiz bir session acalim.

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext
#.config('spark.executer.memory', '16gb') \  -Bu kod satiri kac gb ram kullanacagini belirler. Bu degeri girmediginde kendisi en uygununu secer.
spark = SparkSession.builder \
        .master('local') \
        .appName('pyspark_uygulama') \
        .getOrCreate()

sc = spark.sparkContext  # Burada yukaridaki gibi 'SparkContext' yazinca hata veriyor onun yerine 'sparkContext' boyle yazilmasi gerek.
sc


<SparkContext master=local appName=pyspark_uygulama>

In [14]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[]


# Spark DataFrame

In [25]:
df = spark.read.csv('C:/Users/umutk/Documents/python_calismalar/diabetes.csv', header=True, inferSchema=True)

In [26]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [27]:
type(df)

pyspark.sql.dataframe.DataFrame

**Surekli kullanacagin veriyi tekrar tekrar cekmemek icin '_cache_' kullanarak ram'e tasiyacagiz.**

In [28]:
df.cache()

DataFrame[Pregnancies: int, Glucose: int, BloodPressure: int, SkinThickness: int, Insulin: int, BMI: double, DiabetesPedigreeFunction: double, Age: int, Outcome: int]

In [29]:
df.head(2)

[Row(Pregnancies=6, Glucose=148, BloodPressure=72, SkinThickness=35, Insulin=0, BMI=33.6, DiabetesPedigreeFunction=0.627, Age=50, Outcome=1),
 Row(Pregnancies=1, Glucose=85, BloodPressure=66, SkinThickness=29, Insulin=0, BMI=26.6, DiabetesPedigreeFunction=0.351, Age=31, Outcome=0)]

In [33]:
df.show(2)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 2 rows



In [30]:
df.dtypes

[('Pregnancies', 'int'),
 ('Glucose', 'int'),
 ('BloodPressure', 'int'),
 ('SkinThickness', 'int'),
 ('Insulin', 'int'),
 ('BMI', 'double'),
 ('DiabetesPedigreeFunction', 'double'),
 ('Age', 'int'),
 ('Outcome', 'int')]

In [34]:
df.count()

768

In [35]:
df.columns

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [42]:
df.describe('Glucose').show() # istedigimiz sutunlarin ozetini alabiliyoruz.

+-------+-----------------+
|summary|          Glucose|
+-------+-----------------+
|  count|              768|
|   mean|     120.89453125|
| stddev|31.97261819513622|
|    min|                0|
|    max|              199|
+-------+-----------------+



In [46]:
df.select('Glucose', 'Pregnancies').show(2) # Degisken secip degerlerini almak icin .select() kullaniyoruz.

+-------+-----------+
|Glucose|Pregnancies|
+-------+-----------+
|    148|          6|
|     85|          1|
+-------+-----------+
only showing top 2 rows



In [49]:
df.select('Age').distinct().count() # Age icerisindeki tum unique degerlerin sayisi.

52

In [53]:
df.select('Age').dropDuplicates().count() # Age icerisinde duplicate'leri silip getirdi.

52

In [55]:
# Iki degiskeni caprazlamak istersek.

df.crosstab('Outcome', 'Pregnancies').show()

+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|Outcome_Pregnancies|  0|  1| 10| 11| 12| 13| 14| 15| 17|  2|  3|  4|  5|  6|  7|  8|  9|
+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|                  0| 73|106| 14|  4|  5|  5|  0|  0|  0| 84| 48| 45| 36| 34| 20| 16| 10|
|                  1| 38| 29| 10|  7|  4|  5|  2|  1|  1| 19| 27| 23| 21| 16| 25| 22| 18|
+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+



In [57]:
# Eksik gozlemlere 'dropna()' ya da 'fillna()' uygulamak.

df.fillna(0).show() # Zaten bizde eksik gozlem yok

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|31.0|                   0.248| 26|      1|


## Gozlem secme islemleri

In [58]:
df.filter(df.Age > 45).count()

118

In [60]:
df.groupby('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  268|
|      0|  500|
+-------+-----+



In [61]:
df.groupby('Outcome').agg({'BMI' : 'mean'}).show()

+-------+-----------------+
|Outcome|         avg(BMI)|
+-------+-----------------+
|      1|35.14253731343278|
|      0|30.30419999999996|
+-------+-----------------+



In [68]:
# Yeni bir degisken eklerken

df.withColumn('yeni_degisken', df.BMI / df.Age).select('BMI', 'Age', 'yeni_degisken').show(3)

+----+---+------------------+
| BMI|Age|     yeni_degisken|
+----+---+------------------+
|33.6| 50|             0.672|
|26.6| 31|0.8580645161290323|
|23.3| 32|          0.728125|
+----+---+------------------+
only showing top 3 rows



In [72]:
# Rename column

df.withColumnRenamed('Outcome' , 'Hasta').show(3)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Hasta|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|    1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|    0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|    1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-----+
only showing top 3 rows



In [74]:
# Drop column

df.drop('Glucose').columns

['Pregnancies',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [76]:
# Pandas df cevirme.

pandas_df = df.groupby('Outcome').count().toPandas()

In [78]:
pandas_df

,Outcome,count
0,1,268
1,0,500


# SQL islemleri

In [80]:
df.createOrReplaceTempView('table_df')

In [82]:
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [85]:
spark.sql('select BloodPressure, Insulin from table_df').show(3)

+-------------+-------+
|BloodPressure|Insulin|
+-------------+-------+
|           72|      0|
|           66|      0|
|           64|      0|
+-------------+-------+
only showing top 3 rows



In [87]:
spark.sql('select Outcome, avg(Glucose) from table_df group by Outcome').show()

+-------+------------------+
|Outcome|      avg(Glucose)|
+-------+------------------+
|      1|141.25746268656715|
|      0|            109.98|
+-------+------------------+



In [90]:
sc.stop()